In [ ]:
import json
import random
import string
from datetime import datetime, timedelta
from faker import Faker
import pandas as pd

fake = Faker()

# JSON 포맷의 회원 활동 로그 파일 경로
json_file_path = 'sample_user_log.json'

# 데이터베이스 연결 설정 (필요한 경우 변경)
db_type = 'oracle'  # 'oracle' 또는 'mysql'

if db_type == 'oracle':
    import cx_Oracle
    conn = cx_Oracle.connect('env/admin1234@localhost:1521/xe')
elif db_type == 'mysql':
    import mysql.connector
    conn = mysql.connector.connect(user='root', password='admin1234', host='localhost', database='test')

cursor = conn.cursor()

# JSON 파일에서 회원 정보 추출
user_ids = set()
with open(json_file_path, 'r') as file:
    for line in file:
        log = json.loads(line.strip())
        user_ids.add(log['user_id'])

print(f"총 {len(user_ids)}명의 회원 정보가 생성됩니다.")

# 회원 정보 생성 및 삽입
users = []
for user_id in user_ids:
    user = (
        user_id,  # user_id를 문자열로 유지
        fake.user_name(),
        fake.password(),
        fake.email(),
        fake.phone_number()
    )
    users.append(user)
    
for user in users:
    print(user)
    users.append(user)

if db_type == 'oracle':
    cursor.executemany(
        "INSERT INTO Users (user_id, username, password, email, phone_number) VALUES (:1, :2, :3, :4, :5)",
        users
    )
elif db_type == 'mysql':
    cursor.executemany(
        "INSERT INTO `User` (user_id, username, password, email, phone_number) VALUES (%s, %s, %s, %s, %s)",
        users
    )
conn.commit()

# 상품 정보 생성 및 삽입
brands = [chr(i) for i in range(65, 91)]
products = []
brand_shipping_fees = {}

for brand in brands:
    shipping_fee = random.uniform(5, 20)
    brand_shipping_fees[brand] = shipping_fee

for i in range(100):
    brand = random.choice(brands)
    product = (
        i + 1,
        ''.join(random.choices(string.ascii_uppercase + string.digits, k=10)),
        fake.word(),
        random.choice(['Electronics', 'Clothing', 'Books', 'Home', 'Beauty']),
        random.choice(['Red', 'Blue', 'Green', 'Black', 'White']),
        brands.index(brand) + 1,
        fake.word(),
        random.choice(['S', 'M', 'L', 'XL']),
        random.randint(0, 100)
    )
    products.append(product)

if db_type == 'oracle':
    cursor.executemany(
        "INSERT INTO Product (product_id, product_code,  product_name,  product_type, color, brand_id, option, size, stock) VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9)",
        products
    )
elif db_type == 'mysql':
    cursor.executemany(
        "INSERT INTO Product (product_id, product_code, name, type, color, brand_id, option, size, stock) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
        products
    )
conn.commit()

# 브랜드 정보 생성 및 삽입
brands_data = [(i + 1, brand, brand_shipping_fees[brand]) for i, brand in enumerate(brands)]

if db_type == 'oracle':
    cursor.executemany(
        "INSERT INTO Brand (brand_id, name, shipping_fee) VALUES (:1, :2, :3)",
        brands_data
    )
elif db_type == 'mysql':
    cursor.executemany(
        "INSERT INTO Brand (brand_id, name, shipping_fee) VALUES (%s, %s, %s)",
        brands_data
    )
conn.commit()

# 주문 정보 생성 및 삽입
start_date = datetime(2024, 1, 1)
end_date = datetime.now()
date_range = (end_date - start_date).days

orders = []
order_details = []

for i in range(1000):
    order_date = start_date + timedelta(days=random.randint(0, date_range))
    user_id = random.choice(list(user_ids))
    total_amount = 0
    discount_amount = random.uniform(0, 50)
    shipping_fee = random.uniform(5, 20)
    order_status = random.choice(['Pending', 'Completed', 'Shipped'])
    delivery_address = fake.address()

    order = (
        i + 1,
        int(user_id),  # user_id를 숫자형으로 변환
        order_date,
        total_amount,
        discount_amount,
        shipping_fee,
        order_status,
        delivery_address
    )
    orders.append(order)

    num_items = random.randint(1, 5)
    for _ in range(num_items):
        product_id = random.randint(1, 100)
        quantity = random.randint(1, 3)
        price = random.uniform(10, 500)
        total_amount += price * quantity

        order_detail = (
            len(order_details) + 1,
            i + 1,
            product_id,
            quantity,
            price
        )
        order_details.append(order_detail)

# 주문 정보를 업데이트하여 총 금액을 포함
for order in orders:
    cursor.execute(
        "UPDATE \"Order\" SET total_amount = :1 WHERE order_id = :2" if db_type == 'oracle' else
        "UPDATE `Order` SET total_amount = %s WHERE order_id = %s",
        (order[3], order[0])
    )

if db_type == 'oracle':
    cursor.executemany(
        "INSERT INTO \"Order\" (order_id, user_id, order_date, total_amount, discount_amount, shipping_fee, order_status, delivery_address) VALUES (:1, :2, :3, :4, :5, :6, :7, :8)",
        orders
    )
elif db_type == 'mysql':
    cursor.executemany(
        "INSERT INTO `Order` (order_id, user_id, order_date, total_amount, discount_amount, shipping_fee, order_status, delivery_address) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
        orders
    )
conn.commit()

if db_type == 'oracle':
    cursor.executemany(
        "INSERT INTO OrderDetail (order_detail_id, order_id, product_id, quantity, price) VALUES (:1, :2, :3, :4, :5)",
        order_details
    )
elif db_type == 'mysql':
    cursor.executemany(
        "INSERT INTO OrderDetail (order_detail_id, order_id, product_id, quantity, price) VALUES (%s, %s, %s, %s, %s)",
        order_details
    )
conn.commit()

print("데이터 생성 완료")
cursor.close()
conn.close()
